In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, padding=1),
                        nn.BatchNorm2d(8),
                        nn.ReLU(inplace=True) 
                        )
        self.conv2 = nn.Sequential(
                        nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, padding=1),
                        nn.BatchNorm2d(8),
                        nn.ReLU(inplace=True) 
                        )
        self.conv3 = nn.Sequential(
                        nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, padding=1),
                        nn.BatchNorm2d(8),
                        nn.ReLU(inplace=True),
                        nn.MaxPool2d(kernel_size=2, stride=2)
                        )
        self.conv4 = nn.Sequential(
                        nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=1),
                        nn.BatchNorm2d(16),
                        nn.ReLU(inplace=True)
                        )
        self.conv5 = nn.Sequential(
                        nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1),
                        nn.BatchNorm2d(16) ,
                        nn.ReLU(inplace=True),
                        nn.MaxPool2d(kernel_size=2, stride=2)
                        )
        self.conv6 = nn.Sequential(
                        nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
                        nn.BatchNorm2d(32),
                        nn.ReLU(inplace=True)
                        )
        self.conv7 = nn.Sequential(
                        nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
                        nn.BatchNorm2d(32),
                        nn.ReLU(inplace=True),
                        nn.MaxPool2d(kernel_size=2, stride=2)
                        )
        self.conv8 = nn.Conv2d(32,10,1)
        self.av = nn.AvgPool2d(3)

    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.conv8(x)
        x = self.av(x)
        x = x.view((x.shape[0],-1))
        x = F.log_softmax(x, dim=1)
        return x

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
              ReLU-3            [-1, 8, 28, 28]               0
            Conv2d-4            [-1, 8, 28, 28]             584
       BatchNorm2d-5            [-1, 8, 28, 28]              16
              ReLU-6            [-1, 8, 28, 28]               0
            Conv2d-7            [-1, 8, 28, 28]             584
       BatchNorm2d-8            [-1, 8, 28, 28]              16
              ReLU-9            [-1, 8, 28, 28]               0
        MaxPool2d-10            [-1, 8, 14, 14]               0
           Conv2d-11           [-1, 16, 14, 14]           1,168
      BatchNorm2d-12           [-1, 16, 14, 14]              32
    

In [4]:
torch.manual_seed(1)

batch_size = 512

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomApply([transforms.CenterCrop(22), ], p=0.1),
                        transforms.Resize((28, 28)),
                        transforms.RandomRotation((-15., 15.), fill=0),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


100%|██████████| 9912422/9912422 [00:00<00:00, 91577263.33it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 19696860.78it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 24680925.14it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 5284474.00it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1, verbose=True)

for epoch in range(1, 20+1):
    print(f'Epoch {epoch}')
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Adjusting learning rate of group 0 to 1.0000e-02.
Epoch 1


loss=0.34207627177238464 batch_id=117: 100%|██████████| 118/118 [00:51<00:00,  2.29it/s]



Test set: Average loss: 0.2977, Accuracy: 9258/10000 (93%)

Adjusting learning rate of group 0 to 1.0000e-02.
Epoch 2


loss=0.11330223083496094 batch_id=117: 100%|██████████| 118/118 [00:51<00:00,  2.31it/s]



Test set: Average loss: 0.1132, Accuracy: 9724/10000 (97%)

Adjusting learning rate of group 0 to 1.0000e-02.
Epoch 3


...


Test set: Average loss: 0.0195, Accuracy: 9941/10000 (99%)

Adjusting learning rate of group 0 to 1.0000e-03.
Epoch 19


loss=0.0264126043766737 batch_id=117: 100%|██████████| 118/118 [00:48<00:00,  2.41it/s]



Test set: Average loss: 0.0193, Accuracy: 9944/10000 (99%)

Adjusting learning rate of group 0 to 1.0000e-03.
Epoch 20


loss=0.03865968436002731 batch_id=117: 100%|██████████| 118/118 [00:48<00:00,  2.41it/s]



Test set: Average loss: 0.0195, Accuracy: 9941/10000 (99%)

Adjusting learning rate of group 0 to 1.0000e-03.
